# Импорт

In [1]:
!pip install catboost
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 59.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=17ea5e0b431b5b562a781b001ecdf67336ee6ba0c1e48d00a5e53b7f663bf0ff
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
# при работе через Google Colab
# прописать путь до всех файлов py
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Учёба/8 семестр/диплом/Проект/VKR/Блокноты')

Mounted at /content/drive


In [3]:
import AnalysisFunc
import BaseFuncs
import PredictionSplit
import PredictionResults

ДЛЯ ПРОГНОЗИРОВАНИЙ ДОСТУПНЫ СЛЕДУЮЩИЕ ПОДСЧЕТЫ ОШИБОК/ТОЧНОСТЕЙ:
recall
f1
r2
mae


In [4]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, confusion_matrix, f1_score, classification_report, recall_score, mean_absolute_percentage_error
# from sklearn import preprocessing

import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np


# Загрузка данных

In [5]:
from google.colab import drive
drive.mount('/content/drive')
data = BaseFuncs.GetDataFrameListFromFileConverted("/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/Преобразованные/ПиОА тест div20kr.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# загрузка данных через виндовс
data = BaseFuncs.GetDataFrameListFromFileConverted(BaseFuncs.OpenFileNameWindows())


In [6]:
data.drop_duplicates(keep='last')
data = AnalysisFunc.MakeFloat(data, ["Пол"])

ex = "Контрольная работа 1"

In [7]:
#  найти параметры регрессии
#найти наибольшую глубину
X, Y = PredictionSplit.SplitTestTrainTest(data, ex, ["Пол", "Оценка"]+AnalysisFunc.before)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=27)

estimator=AdaBoostRegressor()
# applying GridSearchCV
parameters_RF = {
     'n_estimators': range(1, 15),
    'learning_rate' : [0.01, 0.05, 0.1, 0.5],
    'loss' : ['linear', 'square', 'exponential'],
}
grid_search=GridSearchCV(cv=5, estimator=estimator, n_jobs=-1,
             param_grid=parameters_RF,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"AdaBoostRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20 )

estimator=RandomForestRegressor()
# applying GridSearchCV
parameters_RF = {'bootstrap': [True], 'max_depth': range(1, 15),
                'max_features': ['sqrt', 'log2', None],
                'min_samples_leaf': [1,2,3],
                'min_samples_split': [1,2,3],
                'n_estimators': range(1, 15)}
grid_search=GridSearchCV(cv=5, estimator=estimator, n_jobs=-1,
             param_grid=parameters_RF,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"RandomForestRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20 )


estimator=DecisionTreeRegressor()
# applying GridSearchCV
parameters_RF={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_depth': range(1, 15),
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1,2,3],
                         'min_samples_split': [1,2,3],
                         'splitter': ['best', 'random']}
grid_search=GridSearchCV(cv=5, estimator=estimator, n_jobs=-1,
             param_grid=parameters_RF,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"DecisionTreeRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20)


estimator=KNeighborsRegressor()
# applying GridSearchCV
parameters_RF = {
    'n_neighbors': range(1,16),
    'leaf_size': range(20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev'),
}
grid_search=GridSearchCV(cv=5, estimator=estimator, n_jobs=-1,
             param_grid=parameters_RF,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"KNeighborsRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20)


Fitting 5 folds for each of 168 candidates, totalling 840 fits
AdaBoostRegressor parameters: {'learning_rate': 0.01, 'loss': 'square', 'n_estimators': 12}
Best Score: -3.453942699586733
Fitting 5 folds for each of 5292 candidates, totalling 26460 fits
RandomForestRegressor parameters: {'bootstrap': True, 'max_depth': 4, 'max_features': 'log2', 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 6}
Best Score: -3.400402267367352
Fitting 5 folds for each of 3024 candidates, totalling 15120 fits
DecisionTreeRegressor parameters: {'criterion': 'absolute_error', 'max_depth': 4, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'splitter': 'random'}
Best Score: -3.4058494195688227
Fitting 5 folds for each of 2400 candidates, totalling 12000 fits
KNeighborsRegressor parameters: {'leaf_size': 34, 'metric': 'minkowski', 'n_neighbors': 14, 'p': 1, 'weights': 'uniform'}
Best Score: -3.461290744689252


In [8]:
#найти наибольшую глубину
X, Y = PredictionSplit.SplitTestTrainPass(data, ex, ["Пол", "Оценка"]+AnalysisFunc.before)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=27)

estimator=AdaBoostClassifier()
# applying GridSearchCV
parameters_RF = {
    'n_estimators':range(1,50),
    'base_estimator':[None, DecisionTreeClassifier(), SVC(), LogisticRegression()],
    'learning_rate': [0.5, 1.0, 2.0, ]
}
grid_search=GridSearchCV(
    estimator=estimator,
    param_grid=parameters_RF)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"AdaBoostClassifier parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_ )

estimator = RandomForestClassifier()
parameters_RF={'max_depth':(1,15, 1),
              'n_estimators':[10,100,200],
              'max_features':[1,3,5,7],
              'min_samples_leaf':[1,2,3],
              'min_samples_split':[1,2,3]
           }
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters_RF,
    scoring ='recall',
    n_jobs = -1,
    cv = 5
)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"RandomForestClassifier parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_ )


estimator = KNeighborsClassifier(algorithm='auto')
parameters_KNN = {
    'n_neighbors': range(1,16),
    'leaf_size': range(20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev'),
}
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters_KNN,
    scoring='recall',
    n_jobs = -1,
    cv = 5
)
pred=grid_search.fit(X_train, y_train)
y_pred_KNN1 =pred.predict(X_test)
print(f"Knn best parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_ )


AdaBoostClassifier parameters: {'base_estimator': LogisticRegression(), 'learning_rate': 1.0, 'n_estimators': 31}
Best Score: 0.8438032061912658
RandomForestClassifier parameters: {'max_depth': 1, 'max_features': 1, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 100}
Best Score: 0.9357098765432099
Knn best parameters: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 11, 'p': 1, 'weights': 'uniform'}
Best Score: 0.8737962962962962


In [9]:
X, Y = PredictionSplit.SplitTestTrainPass(data, ex, ["Пол", "Оценка"]+AnalysisFunc.before)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=27)

estimator = CatBoostClassifier()
parameters_KNN = {
    'iterations': range(1,251, 10),
    'silent': [True],
    'allow_writing_files': [False],
    'learning_rate': [0.01, 0.05, 0.1, 0.5]
}
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters_KNN,
    scoring='recall',
    n_jobs = -1,
    cv = 5
)
pred=grid_search.fit(X_train, y_train)
y_pred_KNN1 =pred.predict(X_test)
print(f"CatBoostClassifier best parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_ )


estimator = CatBoostRegressor()
parameters_KNN = {
    'iterations': range(1,251, 25),
    'silent': [True],
    'depth': range(1, 10, 1),
    'allow_writing_files': [False],
    'learning_rate': [0.01, 0.05, 0.1]
}
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters_KNN,
    scoring='neg_mean_absolute_error',
    n_jobs = -1,
    cv = 5
)
pred=grid_search.fit(X_train, y_train)
y_pred_KNN1 =pred.predict(X_test)
print(f"CatBoostRegressor best parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20 )

CatBoostClassifier best parameters: {'allow_writing_files': False, 'iterations': 51, 'learning_rate': 0.05, 'silent': True}
Best Score: 0.8886728395061727
CatBoostRegressor best parameters: {'allow_writing_files': False, 'depth': 6, 'iterations': 151, 'learning_rate': 0.05, 'silent': True}
Best Score: -4.4282665439451705
